# 02 • Features Builder (табличка/текст/гео/время/изображения)

Цель ноутбука — быстро собрать минимальный набор признаков без CLI/YAML. Все действия управляются флагами `FAST`, `SAFE`, `USE_CACHE`, поэтому ноутбук не пишет ничего лишнего и не трогает диск, кроме кэшей фич-блоков.

Тяжёлые блоки (TF-IDF, geo-neighbors, image embeddings) кэшируются в `artifacts/features/<block>/<key>/…` и поддерживают резюмирование. Запускайте их, когда есть время/ресурсы.

Мини-план тура: до обеда — `num_basic` + `cat_freq` + при необходимости `text_tfidf`; после обеда — `geo_neighbors` и `img_embed`.

In [ ]:
# ——— базовые импорты
import os, sys, json, math, time, gc, warnings
import numpy as np
import pandas as pd
from pathlib import Path

# ——— прогресс/виджеты (но ноут должен работать и без них)
try:
    from tqdm.auto import tqdm
except Exception:
    def tqdm(x, **kw): return x

try:
    import ipywidgets as W
    _WIDGETS = True
except Exception:
    _WIDGETS = False

# ——— наш фич-слой
from common.features import store, assemble
from common.features import (
    num_basic, cat_freq, cat_te_oof, text_tfidf,
    geo_grid, geo_neighbors, time_agg,
    crosses, img_index, img_stats, img_embed
)

from common.cache import make_key
pd.set_option("display.max_colwidth", 120)
warnings.filterwarnings("ignore")

# ——— вспомогалки
def mem_gb(obj=None):
    if obj is None:
        import psutil
        return psutil.Process().memory_info().rss / (1024**3)
    if hasattr(obj, "memory_usage"):
        try:
            return obj.memory_usage(deep=True).sum()/(1024**3)
        except Exception:
            pass
    return np.array(obj).nbytes/(1024**3)

def head(df, n=5):
    display(df.head(n))
    print(df.shape, "mem:", round(mem_gb(df), 3), "GB")

print("Widgets:", _WIDGETS, "| Python:", sys.version)


## Панель параметров

Задаём пути к данным и ключевые колонки. Флаги `FAST`/`SAFE`/`USE_CACHE` управляют тяжёлыми настройками и кэшем. Списки `NUM_COLS`/`CAT_COLS`/`TEXT_COLS`/`MULTI_COLS` можно оставить `None` — сработает авто-детект.

Выбираем, какие блоки включать (минимальный базовый набор уже отмечен). **Важно:** ноутбук ничего не сохраняет автоматически, кроме кэша фич-блоков; сохранение финального набора включается отдельным флагом ниже.

In [ ]:
# ——— БАЗОВЫЕ ПАРАМЕТРЫ (редактируй здесь)
DATA_DIR   = "data"            # можно папку
TRAIN_PATH = f"{DATA_DIR}/train.csv"
TEST_PATH  = f"{DATA_DIR}/test.csv"

ID_COL      = "id"
TARGET_COL  = None             # если есть
DATE_COL    = None
LAT_COL     = None
LON_COL     = None
TEXT_COLS   = None             # например ["text"] или None
MULTI_COLS  = None             # например ["tags"] или None

# явные списки числовых/категориальных (можно оставить None, сделаем авто-детект)
NUM_COLS = None
CAT_COLS = None

# глобальные флаги
FAST      = True    # урезает "тяжёлые" параметры
SAFE      = True    # максимально строгие анти-утечки
USE_CACHE = True    # использовать кэш артефактов фич-блоков

# какой сплит (определится ниже): "kfold" | "group" | "time"
SPLIT_KIND = "kfold"
N_SPLITS   = 5
GROUP_COL  = None      # например user_id/item_id для группового
TIME_EMBARGO = None    # напр. "2D" | "3h" если нужно

# какие блоки включить (по умолчанию базовый минимализм)
ACTIVE_BLOCKS = {
    "num_basic": True,
    "cat_freq": True,
    "cat_te_oof": False,    # включай только если понимаешь анти-утечки
    "text_tfidf": False,
    "geo_grid": False,
    "geo_neighbors": False,
    "time_agg": False,
    "crosses": False,
    "img_stats": False,
    "img_embed": False,     # тяжело; запускать на перерыве/ночью
}

# Сохранение итогового набора (опционально)
SAVE_SET   = False
RUN_TAG    = "exp01"

if _WIDGETS:
    # упрощённые виджеты: переключатели блоков/флагов
    toggles = {k: W.Checkbox(value=v, description=k) for k, v in ACTIVE_BLOCKS.items()}
    flags   = {
        "FAST": W.Checkbox(value=FAST, description="FAST"),
        "SAFE": W.Checkbox(value=SAFE, description="SAFE"),
        "USE_CACHE": W.Checkbox(value=USE_CACHE, description="USE_CACHE")
    }
    display(W.HBox(list(flags.values())))
    display(W.GridBox(list(toggles.values()), layout=W.Layout(grid_template_columns="repeat(3, 220px)")))

    def _read_widgets():
        global FAST, SAFE, USE_CACHE, ACTIVE_BLOCKS
        FAST      = flags["FAST"].value
        SAFE      = flags["SAFE"].value
        USE_CACHE = flags["USE_CACHE"].value
        for k in ACTIVE_BLOCKS:
            ACTIVE_BLOCKS[k] = toggles[k].value

    display(W.Button(description="Применить флаги", button_style="info",
                     tooltip="Считать значения чекбоксов")).on_click(lambda _: _read_widgets())


## Загрузка данных, авто-детект колонок, sanity-чек

Проверяем, что ключевые столбцы на месте, нет странных NaN/констант, и оцениваем типы. Если есть текстовые поля и `TEXT_COLS=None`, лучше задать их явно.

In [ ]:
# ——— чтение
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)

print("Train:", train.shape, "| Test:", test.shape)
assert ID_COL in train.columns and ID_COL in test.columns, "ID_COL не найден"
if TARGET_COL:
    assert TARGET_COL in train.columns, "TARGET_COL не найден в train"

# ——— авто-детект типов
def auto_detect_columns(train, exclude):
    num_cols, cat_cols, text_cols = [], [], []
    for c in train.columns:
        if c in exclude:
            continue
        if pd.api.types.is_numeric_dtype(train[c]):
            num_cols.append(c)
        elif pd.api.types.is_string_dtype(train[c]) and train[c].map(lambda x: isinstance(x, str) and len(x) > 30).mean() > 0.3:
            text_cols.append(c)
        else:
            cat_cols.append(c)
    return num_cols, cat_cols, text_cols

if NUM_COLS is None or CAT_COLS is None or TEXT_COLS is None:
    ex = {ID_COL} | ({TARGET_COL} if TARGET_COL else set())
    n, c, t = auto_detect_columns(train, ex)
    NUM_COLS = n if NUM_COLS is None else NUM_COLS
    CAT_COLS = c if CAT_COLS is None else CAT_COLS
    if TEXT_COLS is None and len(t) > 0:
        TEXT_COLS = t

print("NUM_COLS:", NUM_COLS[:10] if NUM_COLS else [])
print("CAT_COLS:", CAT_COLS[:10] if CAT_COLS else [])
print("TEXT_COLS:", TEXT_COLS)

# ——— базовая очистка: одинаковые столбцы, порядок
train = train.copy()
test = test.copy()
train_cols = [c for c in train.columns if c != TARGET_COL]
test = test[train_cols]  # выравнивание


## Сплиты и анти-утечки

* `time`-сплит — если есть `DATE_COL` и прогноз «в будущее».
* `group` — при утечках через пользователя/товар.
* `kfold` — дефолт, когда нет явных групп/времени.

Все обучаемые трансформы (TE/WOE/CTR) делаются строго **OOF** по выбранным фолдам.

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold

def make_folds(train, task="binary"):
    idx = np.arange(len(train))
    if SPLIT_KIND == "time" and DATE_COL:
        # простой time-сплит: сортировка по времени и нарезка на N_SPLITS чанков
        df = train.sort_values(DATE_COL).reset_index(drop=True)
        fold_sizes = np.full(N_SPLITS, len(df)//N_SPLITS, dtype=int)
        fold_sizes[:len(df)%N_SPLITS] += 1
        cur = 0
        folds = []
        for k, fs in enumerate(fold_sizes):
            val_idx = np.arange(cur, cur + fs)
            tr_idx = np.setdiff1d(np.arange(len(df)), val_idx)
            folds.append((df.index[tr_idx].to_numpy(), df.index[val_idx].to_numpy()))
            cur += fs
        return folds
    elif SPLIT_KIND == "group" and GROUP_COL:
        gkf = GroupKFold(n_splits=N_SPLITS)
        return [(tr, va) for tr, va in gkf.split(idx, groups=train[GROUP_COL].values)]
    else:
        # по умолчанию — KFold (не stratified: у нас может быть регрессия/мульти)
        kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
        return [(tr, va) for tr, va in kf.split(idx)]

FOLDS = make_folds(train)
print("Folds:", len(FOLDS), "| fold sizes:", [len(v) for _, v in FOLDS][:10])


## FeatureStore: как живут пакеты

Каждый блок возвращает `FeaturePackage(name, train, test, kind, cols, meta)` с префиксом имён, чтобы не конфликтовать. Кэш включается флагом `use_cache=USE_CACHE` и записывает артефакты в `artifacts/features/<block>/<key>/…`.

In [ ]:
FS = store.FeatureStore()
BUILT = []  # список имён пакетов в порядке построения

def run_block(name, fn, *args, _include=True, _params=None, **kwargs):
    '''
    Универсальная обёртка: печать параметров, тайминг, кэш, добавление в FS.
    '''
    if not _include:
        print(f"— SKIP {name}")
        return
    t0 = time.time()
    print(f"
=== BUILD {name} ===")
    if _params:
        print("params:", json.dumps(_params, ensure_ascii=False))
    try:
        pkg = fn(*args, **kwargs)
    except Exception as e:
        print(f"{name}: пропущен из-за ошибки: {e}")
        return
    FS.add(pkg)
    BUILT.append(pkg.name)
    dt = time.time() - t0
    ntr = pkg.train.shape[1] if hasattr(pkg.train, "shape") else "?"
    print(f"done {name} in {dt:.1f}s | +{ntr} cols | kind={pkg.kind}")


## Числовые фичи (num_basic)

Импутация, лог-масштабирование (опционально), клиппинг хвостов. В `FAST=True` — без скейла/биннинга. Включайте почти всегда.

In [ ]:
if ACTIVE_BLOCKS["num_basic"] and NUM_COLS:
    params = dict(prefix="num", num_cols=NUM_COLS, log_cols=None, clip_quant=(0.01,0.99),
                  impute="median", scale=None, use_cache=USE_CACHE)
    run_block("num_basic", num_basic.build, train, test, _include=True, _params=params, **params)
else:
    print("num_basic: пропущен (нет NUM_COLS или выключен)")


## Категориальные частоты (cat_freq)

Считает безопасные частоты/доли без таргета — даёт базовый сигнал для high-card категорий. Обычно включаем.

In [ ]:
if ACTIVE_BLOCKS["cat_freq"] and CAT_COLS:
    params = dict(prefix="catf", cat_cols=CAT_COLS, rare_threshold=0.01, use_cache=USE_CACHE)
    run_block("cat_freq", cat_freq.build, train, test, _params=params, **params)
else:
    print("cat_freq: пропущен (нет CAT_COLS или выключен)")


## OOF-кодировки (cat_te_oof)

Target/WOE/CTR-кодировки с анти-утечками по фолдам. Включайте только если понимаете риски; при `FAST=True` лучше отключить.

In [ ]:
if ACTIVE_BLOCKS["cat_te_oof"] and CAT_COLS and TARGET_COL:
    params = dict(prefix="te", cat_cols=CAT_COLS, method="target", smoothing="m-estimate", use_cache=USE_CACHE)
    run_block("cat_te_oof", cat_te_oof.build, train, train[TARGET_COL], test, FOLDS, _params=params, **params)
else:
    print("cat_te_oof: пропущен (нет TARGET_COL/CAT_COLS или выключен)")


## Текстовые фичи (text_tfidf)

TF-IDF по текстовым полям, sparse CSR. В `FAST=True` подними `min_df` и убери SVD. Хорошо работает с линейными моделями.

In [ ]:
if ACTIVE_BLOCKS["text_tfidf"] and TEXT_COLS:
    # берём первый текстовый столбец (либо пробеги циклом)
    text_col = TEXT_COLS[0]
    params = dict(text_col=text_col, min_df=5 if FAST else 2, ngram_range=(1,2), use_char=False,
                  svd_k=None if FAST else 256, prefix="tfidf", use_cache=USE_CACHE)
    run_block("text_tfidf", text_tfidf.build, train, test, _params=params, **params)
else:
    print("text_tfidf: пропущен (нет TEXT_COLS или выключен)")


## Гео-фичи: гриды и плотность

Гриды по 300/1000 м дают хороший prior; соседи (BallTree) тяжелее и зависят от наличия `sklearn`. При `FAST=True` оставьте один крупный грид.

In [ ]:
if ACTIVE_BLOCKS["geo_grid"] and LAT_COL and LON_COL:
    params = dict(lat_col=LAT_COL, lon_col=LON_COL,
                  steps_m=(1000,) if FAST else (300,1000),
                  prefix="geo", use_cache=USE_CACHE)
    run_block("geo_grid", geo_grid.build, train, test, _params=params, **params)
else:
    print("geo_grid: пропущен (нет LAT/LON или выключен)")


In [ ]:
if ACTIVE_BLOCKS["geo_neighbors"] and LAT_COL and LON_COL:
    params = dict(lat_col=LAT_COL, lon_col=LON_COL, radii_m=(1000,) if FAST else (300,1000),
                  prefix="geonb", use_cache=USE_CACHE)
    run_block("geo_neighbors", geo_neighbors.build, train, test, _params=params, **params)
else:
    print("geo_neighbors: пропущен")


## Время: лаги и роллинги

Используйте для временных процессов со строгим «только прошлое»; по возможности добавляйте эмбарго. Всегда следим за анти-утечками.

In [ ]:
if ACTIVE_BLOCKS["time_agg"] and DATE_COL:
    params = dict(date_col=DATE_COL, group_cols=[ID_COL], lags=(1,7), rollings=(7,30),
                  folds=FOLDS if SAFE else None, prefix="time", use_cache=USE_CACHE)
    run_block("time_agg", time_agg.build, train, _params=params, **params)
else:
    print("time_agg: пропущен")


## Взаимодействия (crosses)

Ограниченный набор крестов NUM×NUM и NUM×CAT. Используйте белый список, чтобы не взорвать размерность.

In [ ]:
if ACTIVE_BLOCKS["crosses"]:
    params = dict(whitelist_num_pairs=None, whitelist_num_cat=None, prefix="x", use_cache=USE_CACHE)
    run_block("crosses", crosses.build, train, test, _params=params, **params)
else:
    print("crosses: пропущен")


## Картинки: быстрые статистики vs эмбеддинги

`img_stats` — дешёвый fallback (цвет/контраст/резкость). `img_embed` — мощно, но тяжело; запускайте на перерыве/ночью и опирайтесь на кэш.

In [ ]:
if ACTIVE_BLOCKS["img_stats"]:
    # нужно построить индекс id->список путей (пример — весь набор train+test)
    all_ids = pd.concat([train[ID_COL], test[ID_COL]]).astype(str).unique()
    try:
        id2 = img_index.build_from_dir(Path(DATA_DIR)/"images", all_ids, pattern="{id}/*.jpg", max_per_id=4)
        params = dict(id_col=ID_COL, id_to_images=id2, prefix="imgstats", use_cache=USE_CACHE)
        run_block("img_stats", img_stats.build, train, test, _params=params, **params)
    except Exception as e:
        print("img_stats: ошибка индекса или чтения:", e)
else:
    print("img_stats: пропущен")


In [ ]:
if ACTIVE_BLOCKS["img_embed"]:
    all_ids = pd.concat([train[ID_COL], test[ID_COL]]).astype(str).unique()
    try:
        id2 = img_index.build_from_dir(Path(DATA_DIR)/"images", all_ids, pattern="{id}/*.jpg", max_per_id=4)
        params = dict(
            id_col=ID_COL, id_to_images=id2,
            backbone="resnet50", image_size=224,
            agg="mean", pool="avg", batch_size=64,
            device="auto", precision="auto", dtype="float16",
            prefix="img", use_cache=USE_CACHE
        )
        run_block("img_embed", img_embed.build, train, test, _params=params, **params)
    except Exception as e:
        print("img_embed: пропущен —", e)
else:
    print("img_embed: пропущен")


## Сборка матриц и паспорт

Собираем `X_dense` для GBDT и `X_sparse` для линейки (TF-IDF и др.). Выбираем пакеты по `kind`. Паспорт покажет, сколько фич в каждом пакете, плотность и RAM.

In [ ]:
# разложим добавленные пакеты по типу
dense_pkgs  = [name for name in FS.list() if FS.get(name).kind == "dense"]
sparse_pkgs = [name for name in FS.list() if FS.get(name).kind == "sparse"]

print("DENSE packages:", dense_pkgs)
print("SPARSE packages:", sparse_pkgs)

X_dense_tr, X_dense_te, catalog_dense = (None, None, None)
X_sparse_tr, X_sparse_te, catalog_sparse = (None, None, None)

if len(dense_pkgs):
    X_dense_tr, X_dense_te, catalog_dense = assemble.make_dense(FS, include=dense_pkgs)
    print("Dense shapes:", X_dense_tr.shape, X_dense_te.shape, "| mem:", round(mem_gb(X_dense_tr),3), "GB")

if len(sparse_pkgs):
    X_sparse_tr, X_sparse_te, catalog_sparse = assemble.make_sparse(FS, include=sparse_pkgs)
    print("Sparse shapes:", X_sparse_tr.shape, X_sparse_te.shape)


## Проверки перед сохранением

Убедитесь, что столбцы train/test совпадают, нет NaN/inf (для dense), число строк корректно и объём набора тянет ваша машина.

In [ ]:
from pathlib import Path
import joblib, pickle, json

def save_set(run_tag, Xd_tr, Xd_te, Xs_tr, Xs_te, y=None, folds=None, catalog=None):
    base = Path("artifacts/sets")/run_tag
    base.mkdir(parents=True, exist_ok=True)
    if Xd_tr is not None:
        Xd_tr.to_parquet(base/"X_dense_train.parquet")
        Xd_te.to_parquet(base/"X_dense_test.parquet")
    if Xs_tr is not None:
        from scipy import sparse
        sparse.save_npz(base/"X_sparse_train.npz", Xs_tr)
        sparse.save_npz(base/"X_sparse_test.npz", Xs_te)
    meta = {"catalog": catalog, "rows_train": len(train), "rows_test": len(test), "built": BUILT}
    (base/"meta.json").write_text(json.dumps(meta, ensure_ascii=False, indent=2))
    print("Saved to:", base)

if SAVE_SET:
    cat_merged = catalog_dense or catalog_sparse
    save_set(RUN_TAG, X_dense_tr, X_dense_te, X_sparse_tr, X_sparse_te, catalog=cat_merged)
else:
    print("Сохранение набора выключено (SAVE_SET=False)")


## Что дальше

Открой `notebooks/03_model.ipynb`. Если сохраняли набор, в `03_model` можно читать его с диска; иначе — используйте переменные из текущего kernel.

Шпаргалка: «Если времени мало → включи только num_basic + cat_freq; TF-IDF + линейка даёт быстрый сильный бейзлайн; geo_neighbors и img_embed — запускать на перерыве».